### For Google Colab to import Google Drive

In [ ]:
# from google.colab import drive
# drive.mount("/content/drive")

### Installing Libraries

In [ ]:
#%pip install opencv-python
#%pip install scikit-learn

# Data Preprocessing

### Convert JPG to JPEG

In [ ]:
import cv2
import os
from pathlib import Path
DATA_DIR = "dataset/AI Art vs Real Art/" # Put unzipped files to the unzipped folder and change accordingly

## For Google Colab
# DATA_DIR = "drive/MyDrive/dataset/AI Art vs Real Art/" # Put unzipped files to the unzipped folder and change accordingly

def jpg_to_jpeg(data_dir):
    for dir_name in os.listdir(data_dir):
        files = os.path.join(data_dir,dir_name)
        for filepaths in os.listdir(files):
            file_names = os.path.join(files,filepaths)
            if file_names.endswith(".jpg") or file_names.endswith(".JPG"):
                img = cv2.imread(str(file_names))
                cv2.imwrite(file_names[0:-4]+".jpeg", img)
                os.remove(file_names)

In [ ]:
import imghdr

IMAGE_EXTENSIONS = [".png", ".jpg", ".jpeg"]  # add there all your images file extensions

img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
for filepath in Path(DATA_DIR).rglob("*"):
    if filepath.suffix.lower() in IMAGE_EXTENSIONS:
        img_type = imghdr.what(filepath)
        if img_type is None:
            print(f"{filepath} is not an image")
            os.remove(filepath)
        elif img_type not in img_type_accepted_by_tf:
            print(f"{filepath} is a {img_type}, not accepted by TensorFlow")

### Hyperparameters

In [ ]:
import tensorflow as tf
import keras 

BATCH_SIZE = 64
HEIGHT = 256
WIDTH = 256
IMAGE_SIZE = (HEIGHT, WIDTH)

### Dataset Split

In [ ]:
train_dataset = keras.preprocessing.image_dataset_from_directory(
    label_mode='binary',
    labels='inferred',
    color_mode='rgb',
    directory=DATA_DIR,
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    seed=1337,
    validation_split=0.2,
    subset="training",
)

val_dataset = keras.preprocessing.image_dataset_from_directory(
    label_mode='binary',
    labels='inferred',
    color_mode='rgb',
    directory=DATA_DIR,
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    seed=1337,
    validation_split=0.2, # use 20% as validation
    subset="validation",
)
# The target labels
class_names = train_dataset.class_names
print(class_names)

In [ ]:
import numpy as np
from sklearn.utils import shuffle

# Shuffle the training dataset
train_dataset = train_dataset.shuffle(len(train_dataset), seed=1337)

# Separate images and labels
images = []
labels = []
for image_batch, label_batch in train_dataset:
    images.extend(image_batch)
    labels.extend(label_batch)

# Convert to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Find the class with fewer samples
unique_labels, class_counts = np.unique(labels, return_counts=True)
minority_class = unique_labels[np.argmin(class_counts)]

# Find the indices of the minority class
minority_indices = np.where(labels == minority_class)[0]

# Randomly sample from the majority class
majority_indices = np.where(labels != minority_class)[0]
num_samples_to_keep = len(minority_indices)
majority_indices_sampled = np.random.choice(majority_indices, size=num_samples_to_keep, replace=False)

# Combine indices of both classes
balanced_indices = np.concatenate([minority_indices, majority_indices_sampled])

# Shuffle the combined indices
balanced_indices = shuffle(balanced_indices, random_state=1337)

# Use the balanced indices to create the balanced dataset
balanced_images = images[balanced_indices]
balanced_labels = labels[balanced_indices]

# Create TensorFlow Dataset from balanced data
balanced_train_dataset = tf.data.Dataset.from_tensor_slices((balanced_images, balanced_labels))
balanced_train_dataset = balanced_train_dataset.shuffle(len(balanced_images)).batch(BATCH_SIZE)

# Print class distribution in the balanced dataset
unique_labels, class_counts = np.unique(balanced_labels, return_counts=True)
print("Class distribution in the balanced dataset:")
for label, count in zip(unique_labels, class_counts):
    print(f"Class {label}: {count} samples")

### Display sample images

In [ ]:
import matplotlib.pyplot as plt

def display_binary_images(dataset, class_names):
    plt.figure(figsize=(10, 10))
    for images, labels in dataset.take(1):
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title(class_names[int(labels[i].numpy())])
            plt.axis("off")

display_binary_images(train_dataset, class_names)

# Model Architecture

## Base Model

In [ ]:
NUM_EPOCHS = 20  # Number of epochs for training

In [ ]:
import tensorflow as tf

# Load the VGG16 model pre-trained on ImageNet data
base_model = tf.keras.applications.VGG16(input_shape=(HEIGHT, WIDTH, 3), include_top=False, weights='imagenet')

# Freeze the layers in the base model
base_model.trainable = False

# Add custom classification head
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
predictions = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# Combine the base model with the custom classification head
model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),  # Lower learning rate for fine-tuning
              loss='binary_crossentropy',
              metrics=[keras.metrics.BinaryAccuracy()])


### Util Methods

In [ ]:
# Define Learning Rate Scheduling callback
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',  # Metric to monitor
    factor=0.2,           # Factor by which the learning rate will be reduced
    patience=3,           # Number of epochs with no improvement after which learning rate will be reduced
    verbose=1             # Print message when learning rate is reduced
)

# Define Early Stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Metric to monitor
    patience=5,          # Number of epochs with no improvement after which training will be stopped
    verbose=1,           # Print message when training is stopped
    restore_best_weights=True  # Restore model weights to the point of best validation loss
)



### Model Training

In [ ]:
# Train the model
history = model.fit(balanced_train_dataset,
                    epochs=NUM_EPOCHS,
                    validation_data=val_dataset,
                    callbacks=[lr_scheduler, early_stopping]
                    )


## Fine Tuning #1

In [ ]:
for layer in base_model.layers:
    if layer.name.startswith('conv2') or layer.name.startswith('conv4'):
        layer.trainable = True
    else:
        layer.trainable = False

model.compile(optimizer=keras.optimizers.Adam(1e-6),  # Very low learning rate
              loss='binary_crossentropy',
              metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
# Train the model
history = model.fit(balanced_train_dataset,
                    epochs=NUM_EPOCHS,
                    validation_data=val_dataset,
                    callbacks=[lr_scheduler, early_stopping]
                    )

## Fine Tuning #2

In [ ]:
# # Unfreeze some layers in the base model
# # Here, we unfreeze the last convolutional block
# for layer in base_model.layers:
#     if layer.name.startswith('conv5'):
#         layer.trainable = True
#     else:
#         layer.trainable = False

# Unfreeze specific layers in the base model
# Here, we unfreeze the middle, and last convolutional blocks
for layer in base_model.layers:
    if layer.name.startswith('conv3') or layer.name.startswith('conv5'):
        layer.trainable = True
    else:
        layer.trainable = False

model.compile(optimizer=keras.optimizers.Adam(1e-6),  # Very low learning rate
              loss='binary_crossentropy',
              metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
# Train the model
history = model.fit(balanced_train_dataset,
                    epochs=NUM_EPOCHS,
                    validation_data=val_dataset,
                    callbacks=[lr_scheduler, early_stopping]
                    )

## Fine Tuning #3

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are take into account
model.compile(optimizer=keras.optimizers.Adam(1e-8),  # Very low learning rate
              loss='binary_crossentropy',
              metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
# Train the model
history = model.fit(balanced_train_dataset,
                    epochs=NUM_EPOCHS,
                    validation_data=val_dataset,
                    callbacks=[lr_scheduler, early_stopping]
                    )

# Model Evalutation